## Load PyEpiDAG

In [1]:
import epidag as dag
import numpy as np

## Define a DAG

Compose a script

```
PCore Exp1 {
    # Definitions of nodes
}
```

Then, .. parse to a DAG

In [2]:
script = '''
PCore Exp1 {
    n
    a = 0.5 
    b ~ beta(1, 1)
    c = min([a, b])
    y ~ binom(n, c)
}
'''

js = dag.bn_script_to_json(script)
bn = dag.BayesianNetwork(js)
bn

Name:	Exp1
Nodes:
	n
	b ~ beta(1,1)
	a = 0.5
	c = min([a,b])
	y ~ binom(n,c)

### Single value variable


> VariableName = Number 


In [3]:
SingleValue = bn['a']
print('Node \'a\'')
print(SingleValue)

print('\nFind the value')
print(SingleValue())

Node 'a'
a = 0.5

Find the value
0.5


### Exogenous variable

> VariableName

In [4]:
Exogenous = bn['n']
print('Node \'n\'')
print(Exogenous)

print('\nFind the value; must append external resources')
print(Exogenous({'n': 5}))

Node 'n'
n

Find the value; must append external resources
5


### Random variable

> VariableName ~ p(...)

** p(...) **: a probabilidy density/mass function

In [5]:
Random = bn['b']
print('Node \'b\'')
print(Random)

print('\nSample a value')
print(Random())

Node 'b'
b ~ beta(1,1)

Sample a value
0.340454879615


### Equation output

> OutputName = g(...)

** g(...) **: a mathematical function

In [6]:
Equation = bn['c']
print('Node \'c\'')
print(Equation)

parents = {
    'a': SingleValue(), 
    'b': Random() 
}

print('\nCalculate a value')
print(Equation(parents))

Node 'c'
c = min([a,b])

Calculate a value
0.471975668463


### Pseudo Node
Pseudo nodes are the nodes which can not be implemented in simulation but in relation inference.

> VariableName = f(...)

** f(...) **: a pseudo function start with ** f ** and a list of parent variables follows

In [7]:
## todo

## For simulation model

### Basic case

In [8]:
script = '''
PCore Exp2 {
   mu_x = 0
   mu_y = 10
   sd ~ exp(1)
   x ~ norm(mu_x, sd)
   y ~ norm(mu_y, sd)
}
'''

bn = dag.bn_from_script(script)
sc = dag.as_simulation_core(bn)
sg = sc.generate('exp2')
sg

sd: 0.745943, mu_y: 10, mu_x: 0

In [9]:
y = sg.get_sampler('y')
y

Actor y (norm(mu_y,sd)) on exp2

In [10]:
y(), np.mean([y() for _ in range(1000)])

(9.6319998918248544, 9.9796363313362679)

Intervention

In [11]:
sg.impulse({'mu_y': 100})
sg

sd: 0.745943, mu_y: 10, mu_x: 0

In [12]:
sg['mu_y'] = 100
sg

sd: 0.745943, mu_y: 100, mu_x: 0

In [13]:
y(), np.mean([y() for _ in range(1000)])

(99.010525080137612, 99.987996134278148)

### Define exposed nodes

### Heirarchical node groups

### Random effects

### Example 1: beta-binomial model

In [14]:
# Sample 

In [15]:
# Sample with given nodes

### Example 2:  regression model

### Example 3: BMI model

In [16]:

script = '''
Pcore BMI {
    b0 ~ norm(12, 1)
    b1 = 0.5 # gggg
    pf ~ beta(8, 20)
    foodstore ~ binom(100, pf)
    b0r ~ norm(0, .01)
    ageA ~ norm(20, 3)
    ageB ~ norm(30, 2)
    ps ~ beta(5, 6)
    sexA ~ cat({'m': ps, 'f': 1-ps})
    muA = b0 + b0r + b1*ageA
    bmiA ~ norm(muA, sd)
    sdB = sd * 0.5
    muB = b0 + b0r + b1*ageB
    bmiB ~ norm(muB, sdB)
}
'''


bn = dag.bn_from_script(script)

hie = {
    'country': ['area'],
    'area': ['b0r', 'ps', 'foodstore', 'agA', 'agB'],
    'agA': ['bmiA', 'ageA', 'sexA'],
    'agB': ['bmiB', 'ageB']
}

sc = dag.as_simulation_core(bn, hie,
                            root='country',
                            random=['muA'],
                            out=['foodstore', 'bmiA', 'bmiB'])

pc = sc.generate('Taiwan', {'sd': 1})
pc_taipei = pc.breed('Taipei', 'area')
pc_taipei.breed('A1', 'agA')
pc_taipei.breed('A2', 'agA')
pc_taipei.breed('B1', 'agB')
pc_taipei.breed('B2', 'agB')

ValueError: 

In [17]:
pc

sd: 1, sdB: 0.5, pf: 0.499233, b1: 0.5, b0: 12.5339

### Example 4: A simple agent-based model 

In [30]:
import numpy.random as rd

In [34]:
for i in rd.choice(100, 5):
    print(callable(i))

False
False
False
False
False


In [1]:
import re
import ast
import astunparse

In [25]:
seq = 'max( x(k), y*c, {\'d\': 4}, [2,3, max(4,5)], 7, a=k)'
seq = re.sub(r'\s+', '', seq)
try:
    seq_ast = ast.parse(seq)
except SyntaxError:
    raise SyntaxError
    
f, pars = None, list()
start = False

for s in ast.walk(seq_ast):
    if not start and isinstance(s, ast.Name):
        f = s.id
        start = True
    elif start:
        try:
            eq = astunparse.unparse(s)[:-1]
            pars.append((s, eq))
        except AttributeError:
            break
    
        
print(f)
for p in pars:
    print(p)

max
(<_ast.Call object at 0x00000000052CD940>, 'x(k)')
(<_ast.BinOp object at 0x00000000052CD978>, '(y * c)')
(<_ast.Dict object at 0x00000000052CDA90>, "{'d': 4}")
(<_ast.List object at 0x00000000052CD9E8>, '[2, 3, max(4, 5)]')
(<_ast.Num object at 0x00000000052CD2B0>, '7')
(<_ast.keyword object at 0x00000000052CD128>, 'a=k')


{'arg': 'a', 'value': <_ast.Name at 0x52cd358>}

In [35]:
fn = fn.replace(' ', '')
    mat = re.match(r'(\w+)\((\S+)\)', fn)
    if mat is None:
        raise ValueError

    f, pars = mat.group(1), mat.group(2)

    args = list()
    kw_lock = False
    pars = ','+pars
    while len(pars):
        pars = pars[1:]
        mat = re.match(r'(\w+)=', pars)
        key = None
        if mat:
            kw_lock = True
            key = mat.group(1)
            pars = re.sub(r'\w+=', '', pars, 1)
        else:
            if kw_lock:
                raise ValueError

        # parse value
        for pat in [r'{\S+}', r'\[\S+]', r'[\w\*\+\-\^\/\(\)\*\.]+']:
            mat = re.match(pat, pars)
            if mat:
                try:
                    value = eval(mat.group(0), env, loc)
                except NameError:
                    raise ValueError
                pars = re.sub(pat, '', pars, 1)
                break
        args.append((key, value) if key else value)
    return f, args

False

In [9]:
type(cs[1])

_ast.BinOp